In [ ]:
import pandas as pd
pd.set_option('mode.chained_assignment', None)
import matplotlib.pyplot as plt
import math

import seaborn as sns
sns.set()

In [7]:
stock = 'TSLA'
df = pd.read_csv(f'data/{stock.lower()}_cross_val.csv')
df.date = pd.to_datetime(df.date)
df

date       close    forecast
0   2020-05-21  165.520004  165.520004
1   2020-05-22  163.376007  165.672716
2   2020-05-26  163.774002  163.528464
3   2020-05-27  164.046005  163.929735
4   2020-05-28  161.162003  164.200336
..         ...         ...         ...
249 2021-05-18  577.869995  577.564349
250 2021-05-19  563.460022  578.503380
251 2021-05-20  586.780029  564.197437
252 2021-05-21  580.880005  587.222834
253 2021-05-24  606.440002  581.564060

[254 rows x 3 columns]

# Plotting

In [8]:
plt.figure(figsize=(12, 6))
plt.plot(df['date'], df['close'], label="Actual", color="black")
plt.plot(df['date'], df['forecast'], label="Forecast", color="red", alpha=0.5, linestyle="--")
plt.xlabel('Tid')
plt.ylabel('Aktiepris')
plt.legend()

# Trading

In [9]:
# Setup
last_training = df['close'].values[0]
df = df.iloc[1:] # removing first row as it's training data
periods = len(df)

bank_start = 5000
bank = bank_start

risk_max = 0.25
last_close = df['close'].iloc[-1]

portfolio_array = []

In [10]:
# Simulating trading

stock_count = 0
nr_buys = 0
nr_sells = 0

for row, values in df.iterrows():
    latest_close = last_training if row < 1 else df['close'].iloc[row-1]
    forecast = df['forecast'].iloc[row] if row < periods else latest_close
    expected = forecast - latest_close

    portfolio = stock_count*latest_close
    total_value = portfolio + bank
    portfolio_array.append(total_value)

    max_to_trade = math.floor(total_value * risk_max / latest_close)
    nr_to_buy = max_to_trade if max_to_trade * latest_close < bank else math.floor(bank / latest_close)
    nr_to_sell = max_to_trade if max_to_trade < stock_count else stock_count
        
    if expected > 0:
        bank -= latest_close * nr_to_buy
        stock_count += nr_to_buy
        nr_buys += nr_to_buy
    
    if expected < 0:
        bank += latest_close * nr_to_sell
        stock_count -= nr_to_sell
        nr_sells += nr_to_sell
    
    if row == periods and stock_count > 0: # if last period and holding stocks, sell everything and skip
        bank += latest_close * stock_count
        nr_sells += stock_count

trading_yield = round(bank/bank_start*100 - 100, 2)
index_yield = round((last_close-last_training)/last_training*100, 2)
print(f'Trading yield: {trading_yield}%')
print(f'Index yield: {index_yield}%')

Trading yield: 265.85%
Index yield: 266.38%


In [11]:
df['total_value'] = portfolio_array
df['yield_index'] = 100*(df['close']/last_training - 1) + 100
df['yield_trade'] = 100*(df['total_value']/bank_start - 1) + 100

plt.figure(figsize=(12, 6))
plt.plot(df['date'], df['yield_index'], label="Actual", color="gray")
plt.plot(df['date'], df['yield_trade'], label="Trading", color="orange")
plt.xlabel('Time')
plt.ylabel('Index (start = 100)')
plt.title(f'{stock} one year. Trading {trading_yield}% yield, index {index_yield}% yield')
plt.legend()

print(f'Trading yield: {round(bank/bank_start*100 - 100, 2)}%')
print(f'Index yield: {round((last_close-last_training)/last_training*100, 2)}%')

Trading yield: 265.85%
Index yield: 266.38%
